In [58]:
from functools import partial
import geopandas as gpd
import pandas as pd
import math
import numpy as np
import shapely
from shapely.geometry import Point
import googlemaps
import pgeocode
import pyproj

In [175]:
start_lat, start_lon = (50.4019514, 30.3926109)
disaster_radius_km = 150
flight_radius_km = disaster_radius_km + 200
travel_modes = ["DRIVING", "WALKING"]
travel_mode = travel_modes[0]

CITY_FILE = "cities1000.txt"

In [60]:
# Used to "quickly" limit distance to a certain bounding box so we're not comparing distances for every city on Earth
# Not particularly accurate, but good enough for the purpose
def reverse_haversine(start_location, dist_km, direction="N"):
    dir_lookup = {
        "N": 0,
        "E": math.pi/2,
        "S": math.pi,
        "W": -math.pi/2,
    }
    result = np.radians(start_location)
    lat, long = result
    dist = dist_km / pgeocode.EARTH_RADIUS
    theta = dir_lookup[direction]  # Direction in radians
    
    lat2 = math.asin(
        (math.sin(lat) * math.cos(dist)) + (math.cos(lat) * math.sin(dist) * math.cos(theta))
    )
    long2 = (
        long + math.atan2((math.sin(theta) * math.sin(dist) * math.cos(lat)),
        (math.cos(dist) - (math.sin(lat) * math.sin(lat2))))
    )
    
    return math.degrees(lat2), math.degrees(long2)

In [176]:
loc = (start_lat, start_lon)
bounds = {
    "north": reverse_haversine(loc, flight_radius_km * 2, "N")[0], 
    "east": reverse_haversine(loc, flight_radius_km * 2, "E")[1],
    "south": reverse_haversine(loc, flight_radius_km * 2, "S")[0], 
    "west": reverse_haversine(loc, flight_radius_km * 2, "W")[1],
}

In [177]:
bounds

{'north': 56.69719374845024,
 'east': 40.21174911830772,
 'south': 44.10670905154976,
 'west': 20.57347268169228}

https://download.geonames.org/export/dump/cities1000.zip
http://download.geonames.org/export/dump/readme.txt

In [178]:
city_df = pd.read_csv(
    CITY_FILE, 
    sep="\t", 
    header=0,
    names=[
         "geonameid", 
 "name", 
 "asciiname", 
 "alternatenames", 
 "latitude", 
 "longitude", 
 "feature class", 
 "feature code", 
 "country code", 
 "cc2", 
 "admin1 code", 
 "admin2 code", 
 "admin3 code", 
 "admin4 code", 
 "population", 
 "elevation", 
 "dem", 
 "timezone", 
 "modification date", 

    ]
)

/tmp/ipykernel_269017/2045376136.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  city_df = pd.read_csv(


In [179]:
proj = pyproj.Proj(f"+proj=aeqd +units=km +lat_0={start_lat} +lon_0={start_lon}")
proj_df = gpd.GeoDataFrame(
    city_df, 
    geometry=gpd.points_from_xy(city_df.latitude, city_df.longitude),
    crs=pyproj.CRS("EPSG:4326"),
)


Fast function to limit cities to those that MIGHT be in range

In [180]:
proj_df = proj_df.query(
    f"not (latitude > {bounds['north']} or longitude > {bounds['east']} or latitude < {bounds['south']} or longitude < {bounds['west']})"
)

In [181]:
proj_df.sort_values("longitude")

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date,geometry
107339,785671,Sopot,Sopot,"Opstina Sopot,Opština Sopot,Sopot,Sopot Veligr...",44.51972,20.57361,P,PPLA3,RS,NaN,SE,0,70238,NaN,0,NaN,188,Europe/Belgrade,2012-04-14,POINT (44.51972 20.57361)
98787,763659,Nowe Miasto nad Pilicą,Nowe Miasto nad Pilica,"Nove Mjasto nad Pilicom,Nove Mjasto pie Pilica...",51.61812,20.57619,P,PPLA3,PL,PL,78,1406,140608,NaN,3856,NaN,158,Europe/Warsaw,2010-10-01,POINT (51.61812 20.57619)
98928,766307,Lidzbark Warmiński,Lidzbark Warminski,"Heilsberg,Heilsburg,Kheyl'sberg,Licbark,Lidzba...",54.12588,20.57954,P,PPLA3,PL,NaN,85,2809,280901,NaN,16540,NaN,71,Europe/Warsaw,2014-06-27,POINT (54.12588 20.57954)
109934,549302,Khrabrovo,Khrabrovo,"Chrabroe,Hrabrovo,Khrabrovo,Povunden,Powanden,...",54.89709,20.58035,P,PPL,RU,NaN,23,NaN,NaN,NaN,2143,NaN,9,Europe/Kaliningrad,2016-10-14,POINT (54.89709 20.58035)
107382,787391,Orlovat,Orlovat,"Barlad,Borlod,Orlod,Orlovat,Orlód",45.24171,20.58089,P,PPL,RS,NaN,VO,2,80152,NaN,2159,NaN,76,Europe/Belgrade,2016-10-14,POINT (45.24171 20.58089)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108974,511435,Pereleshinskiy,Pereleshinskiy,"Pereleshinskij,Pereleshinskiy,Perelesjinskij,П...",51.71480,40.19530,P,PPL,RU,NaN,86,NaN,NaN,NaN,3257,NaN,161,Europe/Moscow,2012-01-17,POINT (51.71480 40.19530)
110219,559736,Gornyy,Gornyy,"Gornaya,Gornaya Shoriya,Gorny,Gornyj,Gornyy,Го...",47.81592,40.20363,P,PPL,RU,NaN,61,NaN,NaN,NaN,2716,NaN,187,Europe/Moscow,2012-01-17,POINT (47.81592 40.20363)
110042,553427,Kamenolomni,Kamenolomni,"Kamenolomni,Kamenolomnya,Каменоломни",47.66853,40.20510,P,PPLA2,RU,NaN,61,NaN,NaN,NaN,12262,NaN,62,Europe/Moscow,2013-05-07,POINT (47.66853 40.20510)
109541,535253,Likhoy,Likhoy,"Likhoj,Likhoy,Лихой",48.12662,40.20556,P,PPL,RU,NaN,61,NaN,NaN,NaN,3102,NaN,117,Europe/Moscow,2012-01-17,POINT (48.12662 40.20556)


In [182]:
local_azimuthal_projection = f"+proj=aeqd +units=km +lat_0={start_lat} +lon_0={start_lon}"
wgs84_to_aeqd = partial(
    pyproj.transform,
    pyproj.Proj("EPSG:4326"),
    pyproj.Proj(local_azimuthal_projection),
)

In [183]:
proj_df["geometry"] = proj_df["geometry"].transform(lambda i: Point(wgs84_to_aeqd(i.x, i.y)))

/tmp/ipykernel_269017/1650431364.py:1: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  proj_df["geometry"] = proj_df["geometry"].transform(lambda i: Point(wgs84_to_aeqd(i.x, i.y)))


In [184]:
proj_df["distance"] = proj_df["geometry"].distance(Point(0, 0))

/tmp/ipykernel_269017/3214616330.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  proj_df["distance"] = proj_df["geometry"].distance(Point(0, 0))


In [185]:
proj_df.sort_values("distance")

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,...,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date,geometry,distance
120263,8504651,Sofiyivska Borschagivka,Sofiyivska Borschagivka,"Sofievskaja Borshhagovka,Софиевская Борщаговка",50.41005,30.36724,P,PPL,UA,NaN,...,NaN,NaN,NaN,6571,NaN,163,Europe/Kiev,2013-03-17,POINT (-1.80347 0.90117),2.016086
118318,689487,Vyshneve,Vyshneve,"Vishneve,Vishnevoe,Vishnevoye,Vishnjovoe,Vishn...",50.38913,30.37050,P,PPL,UA,NaN,...,NaN,NaN,NaN,42480,NaN,176,Europe/Kiev,2021-10-20,POINT (-1.57243 -1.42598),2.122718
120265,8504779,Kriukivschina,Kriukivschina,"Krjukivshhina,Krjukovshhina,Крюковщина,Крюківщина",50.37153,30.36861,P,PPL,UA,NaN,...,NaN,NaN,NaN,3509,NaN,177,Europe/Kiev,2021-10-27,POINT (-1.70747 -3.38369),3.790096
120267,8519952,Chabany,Chabany,"Chabani,Chabany,Чабани,Чабаны",50.34071,30.42356,P,PPL,UA,NaN,...,NaN,NaN,NaN,5100,NaN,188,Europe/Kiev,2021-10-27,POINT (2.20320 -6.81180),7.159242
119592,706232,Khotiv,Khotiv,"Khotiv,Khotov,Хотов,Хотів",50.33069,30.46836,P,PPL,UA,NaN,...,NaN,NaN,NaN,4569,NaN,128,Europe/Kiev,2014-01-12,POINT (5.39355 -7.92410),9.585490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107339,785671,Sopot,Sopot,"Opstina Sopot,Opština Sopot,Sopot,Sopot Veligr...",44.51972,20.57361,P,PPLA3,RS,NaN,...,0,70238,NaN,0,NaN,188,Europe/Belgrade,2012-04-14,POINT (-779.91992 -603.82713),986.347951
107365,786624,Rača,Raca,"Raca,Racha,Rača,racha,Рача,راچا",44.22712,20.97754,P,PPLA3,RS,NaN,...,12,71013,NaN,0,NaN,138,Europe/Belgrade,2012-04-16,POINT (-751.87345 -640.21006),987.513343
107517,792830,Batočina,Batocina,"Batochina,Batocina,Batočina,batwchyna,batwtshy...",44.15361,21.08167,P,PPLA3,RS,NaN,...,12,70076,NaN,0,NaN,109,Europe/Belgrade,2012-04-15,POINT (-744.55909 -649.35427),987.941905
107319,785013,Topola,Topola,"Topola,Topola Kragujevacka,Topola Kragujevačka",44.25417,20.68250,P,PPLA3,RS,NaN,...,12,71153,NaN,0,NaN,276,Europe/Belgrade,2012-04-16,POINT (-774.94487 -634.27526),1001.421315


In [186]:
closest_cities = (
    proj_df.query(f"distance > {disaster_radius_km} and distance <= {flight_radius_km} and `feature code` != 'PPL'")
    .sort_values("population", ascending=False)
    .head(20)
)

In [187]:
closest_cities

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,...,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date,geometry,distance
13419,627907,Homyel',Homyel',"GME,Gomel,Gomel',Gomela,Gomelis,Gomel’,Gomeļa,...",52.43450,30.97540,P,PPLA,BY,NaN,...,NaN,NaN,NaN,480951,NaN,138,Europe/Minsk,2019-09-05,POINT (39.64401 226.29061),229.736995
118336,689558,Vinnytsya,Vinnytsya,"VIN,Vinica,Vinicja,Vinitsa,Vinnica,Vinnicja,Vi...",49.23220,28.46871,P,PPLA,UA,NaN,...,689560,NaN,NaN,369839,248.0,260,Europe/Kiev,2022-02-27,POINT (-140.11405 -128.30365),189.983609
118552,692194,Sumy,Sumy,"Soemi,Soemy,Soumy,Sumad,Sumae,Sumai,Sumi,Sumio...",50.92160,34.80029,P,PPLA,UA,NaN,...,NaN,NaN,NaN,294456,NaN,143,Europe/Kiev,2022-02-27,POINT (309.75132 67.01457),316.917706
118897,696643,Poltava,Poltava,"PLV,Paltava,Poltav,Poltava,Poltavae,Poltave,Po...",49.58925,34.55367,P,PPLA,UA,NaN,...,696629,NaN,NaN,288324,156.0,160,Europe/Kiev,2022-03-01,POINT (300.71909 -82.01112),311.701455
119938,710791,Cherkasy,Cherkasy,"CKC,Cerkasad,Cerkasai,Cerkasi,Cerkaso,Cerkassi...",49.44452,32.05738,P,PPLA,UA,NaN,...,710795,NaN,NaN,276360,110.0,111,Europe/Kiev,2020-05-17,POINT (120.72207 -105.14752),160.093158
119602,706369,Khmelnytskyi,Khmelnytskyi,"Chmelnyzkyj,Chmielnicki,HMJ,Hmeljnickij,Khmel'...",49.41835,26.97936,P,PPLA,UA,NaN,...,706375,NaN,NaN,271263,295.0,308,Europe/Kiev,2022-03-08,POINT (-247.58163 -103.74625),268.439839
118820,695594,Rivne,Rivne,"Eractum,RWN,Rivne,Rivno,Rivnė,Riwne,Rouna,Rovn...",50.62308,26.22743,P,PPLA,UA,NaN,...,NaN,NaN,NaN,255106,NaN,207,Europe/Kiev,2022-03-01,POINT (-294.59465 32.86138),296.421790
119579,705812,Kropyvnytskyy,Kropyvnytskyy,"Elisavet,Elisavetgrad,Elizabethgrad,Elizavetgr...",48.50834,32.26618,P,PPLA,UA,NaN,...,705813,NaN,NaN,227413,113.0,114,Europe/Kiev,2022-03-02,POINT (138.44757 -208.87623),250.593309
13472,630468,Babruysk,Babruysk,"Babroejsk,Babruisk,Babrujsk,Babruysk,Bobruisk,...",53.13840,29.22140,P,PPLA2,BY,NaN,...,NaN,NaN,NaN,220517,NaN,159,Europe/Minsk,2012-01-18,POINT (-78.40653 305.09094),315.004869
119437,704147,Kremenchuk,Kremenchuk,"KHU,Kermenchuk,Kramjanchug,Kremenchug,Kremench...",49.06253,33.40484,P,PPLA2,UA,NaN,...,704149,NaN,NaN,220065,90.0,72,Europe/Kiev,2022-03-02,POINT (220.09509 -144.54540),263.315821


In [188]:
closest_cities.shape

(20, 21)

In [190]:
closest = closest_cities.copy()

In [191]:
# mapping and shape utils
import folium
from folium import plugins

# Create Map
map = folium.Map(location=[start_lat,start_lon], zoom_start=7)

start_m = folium.Marker([start_lat, start_lon], popup=(start_lat, start_lon), 
                        icon=folium.Icon(icon='glyphicon glyphicon-fire', color='darkred'))
start_m.add_to(map)

# Plot conflict starting points
for kk, loc in closest.iterrows():
    loc_m = folium.Marker([loc.latitude, loc.longitude], popup=loc['name'], 
                            icon=folium.Icon(icon='glyphicon glyphicon-home', color='blue'))
    loc_m.add_to(map)

# Add fullscreen button
plugins.Fullscreen().add_to(map)
display(map)